In [1]:
from dove.utils.vcf import Vcf

In [9]:
def vcf_to_df(vcf_file):
    with Vcf(vcf_file) as f:
        f.vdf['CSQ'] = f.get_variant_info(concat=True, drop=True)['CSQ']
        f.vdf = f.get_sample_format(concat=True, drop=True)
        vep_header = [i for i in f.meta_info['INFO'] if 'CSQ' in i][0].split('Format: ')[
            1].strip('">').split('|')
        f.vdf.CSQ = f.vdf.CSQ.str.split(',')
        df = f.vdf.explode('CSQ')

    df[vep_header] = df.CSQ.str.split('|', expand=True)
    df.drop(['INFO', 'CSQ'], axis=1, inplace=True)
    df = df[df.PICK == "1"]
    df['GT'] = df['GT'].replace({'0/1': 'Het', '1/1': 'Hom'})
    df['PUBMED'] = df['PUBMED'].str.replace('&', ' ')
    if 'PHENOTYPES' in list(df):
        df['PHENOTYPES'] = df['PHENOTYPES'].str.replace('__', ';').str.replace('&', ' & ').str.replace('_', ' ')
    return df

In [10]:
df = vcf_to_df("/home/bar/Documents/Workbench/pybench/projectBacchus/bacchus/bacchus/media/user_data/2/vcf_upload_bar_2021-01-11_12-53-05-bar/runs/asfd_vep_2021-01-11_12-53-05/2019-368446-full_variant_table_vep.vcf.gz")

In [8]:
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,GT,DP,AD,...,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,GO,PHENOTYPES,ClinVar,ClinVar_CLNSIG,ClinVar_CLNHGVS
0,16,3296983,48115,G,C,10000,PASS,Het,406,"206,199",...,,,,,,,FAMILIAL MEDITERRANEAN FEVER;AUTOSOMAL DOMINAN...,97892,Conflicting_interpretations_of_pathogenicity,NC_000016.9:g.3296983G>C
1,16,3297073,48129,A,G,10000,PASS,Het,405,"206,199",...,,,,,,,Behcet disease & Intermittent hydrarthrosis & ...,36503,Benign,NC_000016.9:g.3297073A>G
2,16,3297166,48162,G,C,10000,PASS,Het,742,"366,376",...,,,,,,,FAMILIAL MEDITERRANEAN FEVER;AUTOSOMAL DOMINAN...,2545,Conflicting_interpretations_of_pathogenicity,NC_000016.9:g.3297166G>C
3,16,3297175,18157361,T,C,10000,PASS,Het,742,"376,366",...,,,,,,,Intermittent hydrarthrosis & Behcet disease & ...,255615,Benign,NC_000016.9:g.3297175T>C
4,16,3297181,48167,C,T,10000,PASS,Het,744,"378,366",...,,,,,,,FAMILIAL MEDITERRANEAN FEVER;AUTOSOMAL DOMINAN...,36500,Benign,NC_000016.9:g.3297181C>T
5,16,3298865,18157836,T,C,10000,PASS,Het,450,"215,235",...,,,,,,,Intermittent hydrarthrosis & Behcet disease & ...,,,
6,16,3299749,48305,G,A,10000,PASS,Het,342,"157,185",...,,,,,,,ClinVar: phenotype not specified & FAMILIAL ME...,36515,Benign,NC_000016.9:g.3299749G>A
7,16,3304567,48496,C,G,10000,PASS,Het,1004,"538,464",...,,,,,,,Intermittent hydrarthrosis & Behcet disease & ...,2543,Conflicting_interpretations_of_pathogenicity,NC_000016.9:g.3304567C>G
8,16,3304573,48499,G,T,10000,PASS,Het,1003,"543,459",...,,,,,,,FAMILIAL MEDITERRANEAN FEVER & ClinVar: phenot...,36511,Benign,NC_000016.9:g.3304573G>T
9,16,3304654,48554,T,C,3041,PASS,Het,415,"244,171",...,,,,,,,Behcet disease & Intermittent hydrarthrosis & ...,36510,Benign,NC_000016.9:g.3304654T>C
